In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
dataframe=pd.read_csv("Training Data 2017 2018.csv")

In [3]:
dataframe=dataframe.drop("Unnamed: 0",axis=1)

In [4]:
from sklearn.utils import shuffle
dataframe_s = shuffle(dataframe)

In [5]:
train=dataframe_s.iloc[:2401,:]
test=dataframe_s.iloc[2402:2835,:]

In [6]:
train=train.dropna()
train=train.drop(columns=['Date'])
train['label']=train['label']+2
test=test.dropna()
test['label']=test['label']+2

In [7]:
from keras.utils.np_utils import to_categorical
y_train=to_categorical(train['label'])
num_class=y_train.shape[1]
print("num_class=",y_train.shape)

C:\Users\qinke\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


num_class= (2398, 5)


In [8]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)

In [9]:
tokenizer.fit_on_texts(train['text'])

In [10]:
x_train=tokenizer.texts_to_matrix(train['text'])
x_test=tokenizer.texts_to_matrix(test['text'])

In [11]:
print(x_train.shape)

(2398, 5000)


In [12]:
print(x_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [13]:
x_train=x_train.reshape(x_train.shape[0],5000,1)
x_train.shape

(2398, 5000, 1)

x_train,x_val,y_train,y_val=train_test_split(x_train,train['label'],test_size=0.10,random_state=42)

In [14]:
x_test=x_test.reshape(x_test.shape[0],5000,1)
x_test.shape

(433, 5000, 1)

In [15]:
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization,Convolution2D, MaxPooling2D
from keras.layers import Dense , Dropout, Lambda, Flatten
from keras import Sequential

In [16]:
from keras.models import Sequential
from keras.layers import Dense
model=Sequential()
model.add(Dense(50,input_shape=(5000, 1)))
#here the standardize is the function we define upwords
model.add(Dense(50,activation='relu'))
model.add(Flatten())
model.add(Dense(5,activation='softmax'))
#only with softmax, it is a linear model

In [17]:
from keras.optimizers import SGD 
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [18]:
print(x_train.shape)
print(y_train.shape)

(2398, 5000, 1)
(2398, 5)


In [20]:
model.fit(x_train,train['label'], epochs=20, batch_size=100)

Epoch 1/20
2398/2398 [==============================] - 13s 6ms/step - loss: 1.5194 - acc: 0.3440
Epoch 2/20
2398/2398 [==============================] - 14s 6ms/step - loss: 1.4682 - acc: 0.3670
Epoch 3/20
2398/2398 [==============================] - 13s 6ms/step - loss: 1.4479 - acc: 0.3837
Epoch 4/20
2398/2398 [==============================] - 14s 6ms/step - loss: 1.4204 - acc: 0.3891
Epoch 5/20
2398/2398 [==============================] - 14s 6ms/step - loss: 1.3625 - acc: 0.4237
Epoch 6/20
2398/2398 [==============================] - 13s 5ms/step - loss: 1.2906 - acc: 0.4775
Epoch 7/20
2398/2398 [==============================] - 13s 5ms/step - loss: 1.1848 - acc: 0.5567
Epoch 8/20
2398/2398 [==============================] - 14s 6ms/step - loss: 1.0424 - acc: 0.6230
Epoch 9/20
2398/2398 [==============================] - 14s 6ms/step - loss: 0.8820 - acc: 0.7048
Epoch 10/20
2398/2398 [==============================] - 15s 6ms/step - loss: 0.7194 - acc: 0.7823
Epoch 11/20
2398/23

In [21]:
model.predict(x_test)

array([[4.3077194e-03, 4.6481395e-01, 4.3460868e-02, 2.4855451e-01,
        2.3886295e-01],
       [4.4322088e-03, 7.8348684e-01, 7.6872513e-02, 1.7269235e-02,
        1.1793912e-01],
       [4.1038085e-02, 2.6107657e-01, 3.3931970e-03, 6.7635900e-01,
        1.8133132e-02],
       ...,
       [2.5211718e-02, 9.5527536e-01, 1.0456561e-05, 1.1803496e-03,
        1.8322160e-02],
       [2.0593813e-01, 1.5398714e-01, 6.4981826e-02, 3.6100093e-01,
        2.1409205e-01],
       [5.0693364e-03, 8.7223998e-05, 1.4117015e-02, 9.6574056e-01,
        1.4985809e-02]], dtype=float32)

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model,to_file='model.png')

In [ ]:
model=Sequential()
model.add(Dense(50,input_shape=(5000, 1)))
#here the standardize is the function we define upwords
model.add(Dense(50,activation='relu'))
model.add(Flatten())
model.add(Dense(5,activation='softmax'))
#only with softmax, it is a linear model

In [ ]:
!pip install pydot

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()

In [ ]:
batch_size = 128 
nb_classes = 5
nb_epoch = 20

In [ ]:
history = LossHistory()
model.fit(X_train, Y_train,
            batch_size=batch_size, nb_epoch=nb_epoch,
            verbose=1, 
            validation_data=(X_test, Y_test),
            callbacks=[history])

In [ ]:
history.loss_plot('epoch')